# **Perform inference**

## **1. Description**

Here, I use the model to perform query questions.

Each question will assess how different compartments of the predation process influence prey capture.

- **Question 1 :** How does expertise influence foraging decisions and prey capture?

- **Question 2 :** How does past experience and memory influence foraging decisions and prey capture?

- **Question 3 :** How does the current interaction influence foraging decision and prey capture?

I will perform approximate inference using sampling to generate probability distributions for my queries.

Each query will return a conditional probability table.

For most queries, we control for the average rank of the prey group (prey skill) as this can influence how predators fare against such prey.

## **2. Prepare the session**

Load the libraries

In [24]:
# To manipulate data
import pandas as pd

# To perform inference
from pgmpy.inference import ApproxInference

# To import the data
from joblib import load
import os

Import the model object.

In [25]:
# Define the parent directory
parent_dir = os.path.abspath(
    os.path.join(os.getcwd(), os.pardir)
)

# Define the subfolder directory
output_folder = os.path.join(
    parent_dir, "outputs"
)

# Name of the model output file
file_name = "BayesNet-fit.joblib"

model = load(
    os.path.join(
        output_folder,
        file_name
    )
)

## **3. Basic queries**

Here, I perform simple queries that will help me assess how to interpret further results.

I start by creating an inference object called "infer".

In [26]:
infer = ApproxInference(model)

### **3.1 Chase rate and prey capture**

First, I want to know if different chase rates are associated with differences in prey capture.

In [ ]:
evid_baseq = {
    "prey_avg_rank_bin": "Q3"
    }

baseq_1 = infer.query(
    variables = ["prey_captured", "chases_persec_bin"],
    evidence = evid_baseq,
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [28]:
print(baseq_1)

+------------------+----------------------------+----------------------------------------+
| prey_captured    | chases_persec_bin          |   phi(prey_captured,chases_persec_bin) |
+==================+============================+========================================+
| prey_captured(1) | chases_persec_bin(Average) |                                 0.4078 |
+------------------+----------------------------+----------------------------------------+
| prey_captured(1) | chases_persec_bin(Low)     |                                 0.3092 |
+------------------+----------------------------+----------------------------------------+
| prey_captured(1) | chases_persec_bin(High)    |                                 0.0866 |
+------------------+----------------------------+----------------------------------------+
| prey_captured(0) | chases_persec_bin(Average) |                                 0.1012 |
+------------------+----------------------------+----------------------------------------+

**Results.**

Average and lower chase rates are associated with greater chances of capturing prey.

**Interpretation.**

This suggests that predators are more efficient when the chase rate is low. Higher chase rates would mean that predators are engaged in longer chases or that they chase often without being efficient.

### **3.2 Foraging mode, chase rate, and prey capture**

In [ ]:
baseq_2 = infer.query(
    variables = ["prey_captured", "chases_persec_bin", "pred_speed_bin"],
    evidence = evid_baseq,
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [30]:
print(baseq_2)

+------------------+----------------------------+-------------------------+-------------------------------------------------------+
| prey_captured    | chases_persec_bin          | pred_speed_bin          |   phi(prey_captured,chases_persec_bin,pred_speed_bin) |
+==================+============================+=========================+=======================================================+
| prey_captured(1) | chases_persec_bin(Average) | pred_speed_bin(Average) |                                                0.3058 |
+------------------+----------------------------+-------------------------+-------------------------------------------------------+
| prey_captured(1) | chases_persec_bin(Average) | pred_speed_bin(Faster)  |                                                0.0336 |
+------------------+----------------------------+-------------------------+-------------------------------------------------------+
| prey_captured(1) | chases_persec_bin(Average) | pred_speed_bin(Slower)  | 

**Results.**

Overall, predators that use the intercepting strategy have higher chances of engaging in fewer or an average number of chases, and these predators have much higher chances of capturing prey.

**Interpretation.**

This suggests that predators using the average strategy are more efficient, and thus, more likely to capture prey. If this is true, then such hunters are probably geared to hunt a greater range of prey with the intercepting strategy.

## **4. Question 1 - Expertise**

Here, the idea is to assess how prey capture changes with hunting repeated atempts. Predators should technically increase their success as they gain experience, which is the acquisition of expertise. I will test how time lags between sessions influence the gain of expertise. 

### **4.1 Foraging mode**

#### **4.1.1 Query 1 - Experience level given time lag**

Here, we ask the question whether the predator's foraging mode changes with experience given the time lag between sessions.

We will perform one query for long time lags, and a second query for shorter time lags.

We start with the query for long time lags.

In [ ]:
evid_q1a = {
    "time_lag_session_log_bin": "Q5",
    "prey_avg_rank_bin": "Q3"
    }

q1a = infer.query(
    variables = ["pred_speed_bin", "xp_level"],
    evidence = evid_q1a,
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

And now the query for short timelags.

In [ ]:
evid_q1b = {
    "time_lag_session_log_bin": "Q1",
    "prey_avg_rank_bin": "Q3"
    }

q1b = infer.query(
    variables = ["pred_speed_bin", "xp_level"],
    evidence = evid_q1b,
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [65]:
print("\nJoint CPD of foraging mode and xp level given long time lags between sessions\n", q1a)
print("\nJoint CPD of foraging mode and xp level given short time lags between sessions\n", q1b)


Joint CPD of foraging mode and xp level given long time lags between sessions
 +-------------------------+--------------------+--------------------------------+
| pred_speed_bin          | xp_level           |   phi(pred_speed_bin,xp_level) |
+=========================+====================+================================+
| pred_speed_bin(Average) | xp_level(advanced) |                         0.2068 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Average) | xp_level(interm)   |                         0.3452 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Average) | xp_level(novice)   |                         0.1682 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(Slower)  | xp_level(advanced) |                         0.0652 |
+-------------------------+--------------------+--------------------------------+
| pred_speed_bin(S

**Results**.

Overall, it seems that at any experience level and regardless of the time lag between sessions (i.e., low or high time lags), predators are more likely to use an intercepting foraging mode. Chances are also higher that they use such strategy when they are intermediate hunters.

**Interpretation**. 

These results show that the intercepting foraging mode (population average) is more likely to be used than the strategies at the end of the spectrum. Moreover, our results suggest that the chances of using a specific foraging mode is not influenced by the time lag between sessions.

Because predators are more likely to use the intercepting mode when they are intermediate hunters, it is possible that this results from them adjusting to a greater range of prey being encountered at this level of experience. Thus, this strategy could act as a buffer for a greater spectrum of prey.



### **4.2 Chase rate**

#### **4.2.1 Query 2 - Experience level given time lag**

We now perform a similar query, but looking at the predator's chase rate. The hypothesis is that higher experience should lead to more chases. Alternatively, lower chase rates with experience may indicate higher performance because of optimal strategies. We will adress this second prediction later in query 3.

In [ ]:
evid_q2a = {
    "time_lag_session_log_bin": "Q5",
    "prey_avg_rank_bin": "Q3"
    }

q2a = infer.query(
    variables = ["chases_persec_bin", "xp_level"],
    evidence = evid_q2a,
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

Now fort short time lags

In [ ]:
evid_q2b = {
    "time_lag_session_log_bin": "Q1",
    "prey_avg_rank_bin": "Q3"
    }

q2b = infer.query(
    variables = ["chases_persec_bin", "xp_level"],
    evidence = evid_q2b,
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [64]:
print("\nJoint CPD of chase rate and xp level given long time lags between sessions\n", q2a)
print("\nJoint CPD of chase rate and xp level given short time lags between sessions\n", q2b)


Joint CPD of chase rate and xp level given long time lags between sessions
 +----------------------------+--------------------+-----------------------------------+
| chases_persec_bin          | xp_level           |   phi(chases_persec_bin,xp_level) |
+============================+====================+===================================+
| chases_persec_bin(Average) | xp_level(advanced) |                            0.1642 |
+----------------------------+--------------------+-----------------------------------+
| chases_persec_bin(Average) | xp_level(interm)   |                            0.2604 |
+----------------------------+--------------------+-----------------------------------+
| chases_persec_bin(Average) | xp_level(novice)   |                            0.1114 |
+----------------------------+--------------------+-----------------------------------+
| chases_persec_bin(High)    | xp_level(advanced) |                            0.0386 |
+----------------------------+-------------

**Results**.

Similar to the foraging mode, chances are higher that predators will have average chase rates when they are intermediate hunters, but also regardless of their experience level. Interestingly, there are also higher chances that chase rates will be low when predators are intermediate and advanced.

**Interpretation**.

Given that the probability of having high chase rates is low at any level, it seems that predators quickly become better at optimizing chase rates, regardless of the time lag between sessions. For the more efficient predators (low and average chase rates), it remains more likely that lower/average chase rates will be observed when predators are intermediate hunters. This could be due to :

- Advanced predators encounter prey that are more experienced and thus won't necessarily be as efficient

- There is not as much data for advanced predators compared to novice predators

### **4.3 Prey capture**

#### **4.3.1 Query 3 - Chase rate given experience level**

We now perform queries for the joint probability of prey capture and chase rate given different experience levels.

In [ ]:
evid_q3a = {
    "prey_avg_rank_bin": "Q3",
    "xp_level": "novice"
    }

q3a = infer.query(
    variables = ["prey_captured", "chases_persec_bin"],
    evidence = evid_q3a,
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

We now perform the same query but for when predators are advanced.

In [ ]:
evid_q3b = {
    "prey_avg_rank_bin": "Q3",
    "xp_level": "advanced"
    }

q3b = infer.query(
    variables = ["prey_captured", "chases_persec_bin"],
    evidence = evid_q3b,
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [63]:
print("\nJoint CPD of prey captured and given chase rate given *novice* xp level:\n", q3a)
print("\nJoint CPD of prey captured and given chase rate given *advanced* xp level:\n", q3b)


Joint CPD of prey captured and given chase rate given *novice* xp level:
 +------------------+----------------------------+----------------------------------------+
| prey_captured    | chases_persec_bin          |   phi(prey_captured,chases_persec_bin) |
+==================+============================+========================================+
| prey_captured(1) | chases_persec_bin(Average) |                                 0.3810 |
+------------------+----------------------------+----------------------------------------+
| prey_captured(1) | chases_persec_bin(Low)     |                                 0.3470 |
+------------------+----------------------------+----------------------------------------+
| prey_captured(1) | chases_persec_bin(High)    |                                 0.0756 |
+------------------+----------------------------+----------------------------------------+
| prey_captured(0) | chases_persec_bin(Average) |                                 0.0992 |
+--------------

**Results.**

**Interpretation.**

**Notes:** This result could be combined with prey speed to test two processes simultaneously

## **5. Question 2 - Past experience and memory**

### **5.1 Foraging mode**

#### **5.1.1 Query 4 - Range of prey encountered**

In [ ]:
q4 = infer.query(
    variables = ["pred_speed_bin", "iqrange_prey_speed_bin"],
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [67]:
print("\nJoint CPD of the predator's foraging mode and range of prey encountered\n", q4)


Joint CPD of the predator's foraging mode and range of prey encountered
 +-------------------------+----------------------------+----------------------------------------------+
| pred_speed_bin          | iqrange_prey_speed_bin     |   phi(pred_speed_bin,iqrange_prey_speed_bin) |
+=========================+============================+==============================================+
| pred_speed_bin(Faster)  | iqrange_prey_speed_bin(Q5) |                                       0.0240 |
+-------------------------+----------------------------+----------------------------------------------+
| pred_speed_bin(Faster)  | iqrange_prey_speed_bin(Q4) |                                       0.0164 |
+-------------------------+----------------------------+----------------------------------------------+
| pred_speed_bin(Faster)  | iqrange_prey_speed_bin(Q3) |                                       0.0164 |
+-------------------------+----------------------------+--------------------------------------

**Results.**

The probability of adopting an intercepting foraging mode is much higher than other modes regardless of the range of prey encountered.

Let's see if the foraging mode changes with past success.

#### **5.1.2 Query 5 - Prior prey capture**

In [ ]:
q5 = infer.query(
    variables = ["pred_speed_bin", "prior_prey_captured"],
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [69]:
print("\nJoint CPD of foraging mode and prior prey capture\n", q5)


Joint CPD of foraging mode and prior prey capture
 +-------------------------+--------------------------+-------------------------------------------+
| pred_speed_bin          | prior_prey_captured      |   phi(pred_speed_bin,prior_prey_captured) |
+=========================+==========================+===========================================+
| pred_speed_bin(Faster)  | prior_prey_captured(1.0) |                                    0.0638 |
+-------------------------+--------------------------+-------------------------------------------+
| pred_speed_bin(Faster)  | prior_prey_captured(0.0) |                                    0.0258 |
+-------------------------+--------------------------+-------------------------------------------+
| pred_speed_bin(Slower)  | prior_prey_captured(1.0) |                                    0.1418 |
+-------------------------+--------------------------+-------------------------------------------+
| pred_speed_bin(Slower)  | prior_prey_captured(0.0) |   

**Results.**

Chances of adopting an intercepting strategy are much higher when predators previously captured prey. This is because at the population level, it is the optimal strategy. Adopting an ambush (slower) strategy is more rare (14%) when prey were previously captured, but 2.2x more likely than the cursorial strategy (fast)

When no captures occured, predators are also more likely to adopt the intercepting strategy, probably because it is the best at managing any type of prey overall.

#### **5.1.3 Query 6 - Prior chase rate**

The prior chase rate is the highest scoring variable influencing foraging mode. Let us investigate this with the next query.

In [ ]:
q6 = infer.query(
    variables = ["pred_speed_bin", "chases_persec_bin"],
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [72]:
print("\nJoint CPD of foraging mode and prior chase rate\n", q6)


Joint CPD of foraging mode and prior chase rate
 +-------------------------+----------------------------+-----------------------------------------+
| pred_speed_bin          | chases_persec_bin          |   phi(pred_speed_bin,chases_persec_bin) |
+=========================+============================+=========================================+
| pred_speed_bin(Faster)  | chases_persec_bin(Low)     |                                  0.0296 |
+-------------------------+----------------------------+-----------------------------------------+
| pred_speed_bin(Faster)  | chases_persec_bin(High)    |                                  0.0130 |
+-------------------------+----------------------------+-----------------------------------------+
| pred_speed_bin(Faster)  | chases_persec_bin(Average) |                                  0.0470 |
+-------------------------+----------------------------+-----------------------------------------+
| pred_speed_bin(Slower)  | chases_persec_bin(Low)     |   

**Results.**

Similar to what we got above, chances are much greater that a predator will adopt an intercepting foraging mode when the prior chase rate either low or average.